In [6]:
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
bond  = pd.read_csv("../data/raw/Isin.csv")

In [8]:
bond.columns

Index(['IsinIdx', 'TickerIdx', 'ActualMaturityDateKey', 'IssueDateKey',
       'Seniority', 'Currency', 'ActivityGroup', 'Region', 'Activity',
       'RiskCaptain', 'Owner', 'CompositeRating', 'IndustrySector',
       'IndustrySubgroup', 'MarketIssue', 'IssuedAmount', 'CouponType'],
      dtype='object')

In [9]:
bond.head(2)

,IsinIdx,TickerIdx,ActualMaturityDateKey,IssueDateKey,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,IndustrySubgroup,MarketIssue,IssuedAmount,CouponType
0,0,238,20381231,20051129,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,1.246002e+09,STEP CPN
1,1,238,20331231,20051129,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,4.901086e+09,FIXED


In [10]:
bond.groupby("MarketIssue").size()

MarketIssue
Canadian              2
Domestic            788
Domestic mtn        640
Euro mtn           7374
Euro non-dollar    1654
Euro-dollar        3553
Euro-zone          2001
Global             5690
Priv placement     3146
UK gilt stock        43
US domestic        2216
US govt             169
US non-dollar         4
Yankee              114
dtype: int64

In [11]:
bond.loc[~bond.Seniority.isin(["GOV","SEC","SEN","SUB"]), "Seniority"] = "OTHER"
bond.loc[~bond.Activity.isin(["Asia","RETAIL","GBP SAS", "ARGENTINIA"]), "Activity"] = "OTHER"
bond.loc[bond.Currency.isin(
    (bond.groupby("Currency").size())[bond.groupby("Currency").size()<350].index.tolist()
), "Currency"] = "OTHER"

bond.loc[bond.CompositeRating.str.contains("C|D"), "CompositeRating"] = "LOWER"
# Risk captain
bond.loc[bond.RiskCaptain.isin(
    (bond.groupby("RiskCaptain").size())[bond.groupby("RiskCaptain").size()<200].index.tolist()
), "RiskCaptain"] = "OTHER"

In [12]:
data = pd.read_csv("/Users/hauptjoh/Projects/DSG_2018_Round1/data/prepared/data_basic_1.csv", compression = "gzip")

/Users/hauptjoh/anaconda/envs/deeplearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
tmp = data.loc[data.CustomerInterest == 1]

In [14]:
tmp = tmp.loc[tmp.Week<61]

In [15]:
tmp.columns

Index(['PredictionIdx', 'CustomerIdx', 'IsinIdx', 'BuySell',
       'CustomerInterest', 'Week'],
      dtype='object')

In [16]:
full = tmp.merge(bond, on=["IsinIdx"], how="left")

In [17]:
final_table = pd.DataFrame({"CustomerIdx":full.CustomerIdx.unique()})

In [27]:
bondfeature_list = ["Seniority","Currency","ActivityGroup","Region","Activity", 
                    "RiskCaptain", "CompositeRating", "IndustrySector",
                   "MarketIssue"]

In [ ]:
for var in bondfeature_list:
    count_table = full.groupby(["CustomerIdx",var]).size().to_frame('count').reset_index()

    #count_table = count_table.merge(
    #    count_table.groupby("CustomerIdx").max().drop(var, axis=1).reset_index(), 
    #                  on =["CustomerIdx"], how="left")

    count_table.sort_values(["CustomerIdx", "count"], ascending=False).groupby("CustomerIdx").head(1)
    count_table = count_table.loc[:, ["CustomerIdx",var]]
    
    final_table = final_table.merge(count_table, on=["CustomerIdx"], how = "left")
    del(count_table)

In [ ]:
final_table

In [ ]:
final_table.isna().sum()

In [ ]:
final_table.to_csv("/Users/hauptjoh/Projects/DSG_2018_Round1/data/prepared/favorite_bonds_w1-60.csv", index=False, compression = "gzip")